In [0]:
!pip install -q tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 27kB/s 
     |████████████████████████████████| 3.9MB 42.2MB/s 
     |████████████████████████████████| 450kB 53.1MB/s 
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [0]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers

In [0]:
# CIFAR is a set of 60k images 32 x 32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

# constant 
BATCH_SIZE = 128
EPOCHS = 20
CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = tf.keras.optimizers.RMSprop()

In [0]:
# define the covenet
def build(input_shape, classes):
  model = models.Sequential()
  model.add(layers.Convolution2D(32, (3, 3), activation='relu', input_shape=input_shape))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))

  model.add(layers.Flatten())
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(classes, activation='softmax'))
  return model

In [0]:
# data: shuffle and split between train and test sets
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
# normalize 
X_train, X_test = X_train/255.0, X_test/255.0

In [0]:
# convert to categorical
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, CLASSES)

In [0]:
model = build((IMG_ROWS, IMG_COLS, IMG_CHANNELS), CLASSES)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 7200)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               3686912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5

In [0]:
%load_ext tensorboard
import datetime

In [0]:
from tensorflow.keras.callbacks import TensorBoard

In [0]:
log_dir = 'logs/fit/' + datetime.datetime.now(). strftime('%Y%m%d - %H%M%S')
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [0]:
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])

In [0]:
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VALIDATION_SPLIT, verbose=VERBOSE, callbacks=[tensorboard_callback])

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 55s 1ms/sample - loss: 1.7035 - accuracy: 0.3971 - val_loss: 1.3796 - val_accuracy: 0.5026
Epoch 2/20
40000/40000 [==============================] - 55s 1ms/sample - loss: 1.3638 - accuracy: 0.5187 - val_loss: 1.3195 - val_accuracy: 0.5299
Epoch 3/20
40000/40000 [==============================] - 55s 1ms/sample - loss: 1.2361 - accuracy: 0.5663 - val_loss: 1.3186 - val_accuracy: 0.5450
Epoch 4/20
40000/40000 [==============================] - 56s 1ms/sample - loss: 1.1421 - accuracy: 0.5994 - val_loss: 1.0941 - val_accuracy: 0.6207
Epoch 5/20
40000/40000 [==============================] - 55s 1ms/sample - loss: 1.0738 - accuracy: 0.6238 - val_loss: 1.0730 - val_accuracy: 0.6313
Epoch 6/20
40000/40000 [==============================] - 55s 1ms/sample - loss: 1.0137 - accuracy: 0.6469 - val_loss: 1.0995 - val_accuracy: 0.6153
Epoch 7/20
40000/40000 [==============================] 

In [0]:
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)

10000/10000 [==============================] - 4s 357us/sample - loss: 1.0485 - accuracy: 0.6860


In [0]:
print("Test Score: ", score[0])
print("Test Accuracy: ", score[1])

Test Score:  1.0485174385070801
Test Accuracy:  0.686
